In [1]:
from lensid.utils.ml_utils import *

In [2]:
indir = 'train'
data_dir = '../../data/qts/'+str(indir)+'/'

In [3]:
df_lensed = pd.read_csv('../../data/dataframes/'+str(indir)+'/lensed.csv' )
df_lensed=df_lensed.drop(columns=['Unnamed: 0'])
df_lensed['img_0']=df_lensed['img_0'].values 
df_lensed['img_1']=df_lensed['img_1'].values 
df_lensed=df_lensed[:2800]
df_unlensed = pd.read_csv('../../data/dataframes/'+str(indir)+'/unlensed_half.csv' )
df_unlensed=df_unlensed.drop(columns=['Unnamed: 0'])
df_unlensed = df_unlensed.sample(frac = 1,random_state = 42).reset_index(drop = True)[:2800]
df_train = pd.concat([df_lensed,df_unlensed],ignore_index = True)
df_train=df_train.sample(frac = 1).reset_index(drop = True)

In [ ]:
det='H1'
X , y,missing_ids, df_train =  generate_resize_densenet_fm(df_train).DenseNet_input_matrix(det = det,data_mode_dense="current",data_dir=data_dir,phenom=True)

In [ ]:
cv = StratifiedKFold(n_splits = 4, shuffle = True ,random_state = 42)

for i ,(train,test) in enumerate(cv.split(X,y)):
    print(train.shape,test.shape)

In [ ]:
plt.rcParams["figure.figsize"] = (10,10)

    
tprs = []
aucs = []
mean_fpr = 10**np.linspace(-3,0,10)


fig,ax = plt.subplots()
for i ,(train,test) in enumerate(cv.split(X,y)):

    classifier = train_densenet(X[train],y[train],det,10,0.01)#0.005, 0.01
    classifier.save('dense_out/'+det+'_'+str(i)+'.h5')
    #classifier = load_model(det+'_'+str(i)+'.h5')
    probs = classifier.predict_classes(X[test])

    fpr, tpr, threshold = metrics.roc_curve(y[test], probs)
    roc_auc = metrics.auc(fpr, tpr)

    ax.plot(fpr, tpr, 'm',linestyle= "-", label = "ROC fold { "+ str(i)+" } (AUC = " + str(roc_auc)+ " )",lw =3,alpha=0.5)
    interp_tpr = np.interp(mean_fpr,fpr,tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(roc_auc)

mean_tpr = np.mean(tprs,axis = 0)
mean_tpr[-1]=1.0
mean_auc = auc(mean_fpr,mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr,mean_tpr,color='b',label = r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc , std_auc),lw = 2,alpha=.8)
    
std_tpr = np.std(tprs,axis=0)
tprs_upper = np.minimum(mean_tpr +std_tpr,1)
tprs_lower = np.maximum(mean_tpr-std_tpr,0)
ax.fill_between(mean_fpr,tprs_lower,tprs_upper,color="grey",alpha=.5,label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[1e-3,1],ylim=[1e-3,1.05],
          title = "ROC - Error Bars H1 QTs",xscale='log',yscale='log')
plt.savefig('ROC-log-QTs-'+det+'.png')

ax.legend(loc ="lower right")
plt.show()